In [1]:

# Default locations if parameter not passed
benchmark_data_path = "/scratch/rodrigo.freitas/remy-schedulers/tb-compare.pkl"
output_folder = "/scratch/rodrigo.freitas/remy-schedulers/output-tb-test"


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
from rich import print as rprint
import seaborn as sns
import os
from scipy.stats import bootstrap

benchmark_data = pickle.load(open(benchmark_data_path, "rb"))

df = pd.DataFrame(benchmark_data["dataframe"])

# create the output folder if it does not exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)


rprint(df.columns)
rprint(df.head())
rprint(df.tail())

Index(['name', 'hosts', 'type', 'iter', 'node_count', 'full_output', 'tb_path',
       'tb-time', 'image', 'time'],
      dtype='object')

name                    hosts type  iter  node_count  \
0  tb-compare  sorgan-cpu1,sorgan-cpu2  fft  1000           1   

                                         full_output         tb_path  \
0  [1749528216.276838] [sorgan-cpu1:153370:0]    ...  tb-tdg-no-loop   

    tb-time           image      time  
0  0.163592  improv-144.sif  0.668982

name                    hosts type  iter  node_count  \
0  tb-compare  sorgan-cpu1,sorgan-cpu2  fft  1000           1   

                                         full_output         tb_path  \
0  [1749528216.276838] [sorgan-cpu1:153370:0]    ...  tb-tdg-no-loop   

    tb-time           image      time  
0  0.163592  improv-144.sif  0.668982

In [3]:
sns.set_theme()

colors = [
    "#7f7f7f",  # grey
    "#621dac",  # main purple
    "#c5702d",  # orange
    "#000000",  # black,
    "#099892",  # teal
    "#ffd400",  # yellow
    "#7e57c4",  # pink/purple,
]
colors_backup = colors.copy()

In [4]:
def calculate_bootstrap_error(data):
    n_runs = benchmark_data["config"].metadata.runs
    if n_runs < 2:
        return 0
    res = bootstrap(
        (data,), np.mean, confidence_level=0.95, n_resamples=1000, method="basic"
    )
    return res.standard_error

In [5]:
bench_names = df["name"].unique()
run_metadata = benchmark_data["config"].applications

for bench in bench_names:
    bench_metadata = run_metadata[bench]
    n_runs = benchmark_data["config"].metadata.runs
    print(f"Benchmark: {bench} - Runs: {n_runs}")
    # print(bench_metadata)
    df_bench = df[df["name"] == bench]
    for idx, row in df_bench.iterrows():
        print(f"--- Row {idx} ---")
        for col, val in row.items():
            print(f"{col}: {val}")   # printing val directly never truncates
        print()




Benchmark: tb-compare - Runs: 1
--- Row 0 ---
name: tb-compare
hosts: sorgan-cpu1,sorgan-cpu2
type: fft
iter: 1000
node_count: 1
full_output: [1749528216.276838] [sorgan-cpu1:153370:0]        ib_verbs.h:87   UCX  ERROR ibv_exp_query_device(mlx5_0) returned 95: Operation not supported
Running Task Benchmark
  Configuration:
    Task Graph 1:
      Time Steps: 32
      Max Width: 32
      Dependence Type: fft
      Radix: 3
      Period: 0
      Fraction Connected: 0.250000
      Kernel:
        Type: compute_bound
        Iterations: 1000
        Samples: 16
        Imbalance: 0.000000
      Output Bytes: 16
      Scratch Bytes: 0
Interpolation : 25137
Scheduling : 41935
Loop 0 : Elapsed time : 0.101880
Loop 1 : Elapsed time : 0.013889
Loop 2 : Elapsed time : 0.013900
Loop 3 : Elapsed time : 0.014167
Loop 4 : Elapsed time : 0.014145
Total Tasks 1024
Total Dependencies 2666
Total FLOPs 131137536
Total Bytes 0
Elapsed Time 1.635921e-01 seconds
FLOP/s 8.016129e+08
B/s 0.000000e+00


tb_pat